<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/YOLOv4_Tensorflow_modifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 將Colab與google_drive綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 從Github下載yolo-v4-tf.keras資料庫

!git clone https://github.com/taipingeric/yolo-v4-tf.keras.git

Cloning into 'yolo-v4-tf.keras'...
remote: Enumerating objects: 1377, done.
remote: Total 1377 (delta 0), reused 0 (delta 0), pack-reused 1377
Receiving objects: 100% (1377/1377), 29.82 MiB | 32.98 MiB/s, done.
Resolving deltas: 100% (714/714), done.


In [ ]:
import sys
FOLDER_PATH = '/content/yolo-v4-tf.keras' 
sys.path.append(FOLDER_PATH)